In [6]:
import numpy as np
import pymc as pm
import arviz as az


reclama = np.array([1.5, 2.0, 2.3, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0,
                    6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0])
vanzari = np.array([5.2, 6.8, 7.5, 8.0, 9.0, 10.2, 11.5, 12.0, 13.5, 14.0,
                    15.0, 15.5, 16.2, 17.0, 18.0, 18.5, 19.5, 20.0, 21.0, 22.0])

# pt predictii
new = np.array([2.5, 5.0, 9.0])

with pm.Model() as model_bayes:
    x = pm.Data("x_pub", reclama)
    
    
    intercept = pm.Normal("intercept", mu=0, sigma=10)
    slope = pm.Normal("slope", mu=0, sigma=5)
    
    
    mu = intercept + slope * x

    sigma = pm.HalfNormal("dispersie", sigma=5)  # zgomot
    
    # Likelihood
    pm.Normal("y_obs", mu=mu, sigma=sigma, observed=vanzari)
    
    posterior = pm.sample(2000, tune=1000, target_accept=0.9, random_seed=42)


print("Rezumat")
print(az.summary(posterior, var_names=["intercept", "slope"]))


hdi_coef = az.hdi(posterior, var_names=["intercept", "slope"], hdi_prob=0.94)
print("\nCoefficient HDI 94%")
print(hdi_coef)


with model_bayes:
    ppc_pred = pm.sample_posterior_predictive(
        posterior,
        var_names=["y_obs"],
        predictions={"x_pub": new},
        random_seed=42
    )


pred_array = ppc_pred.predictions["y_obs"].stack(samples=("chain", "draw")).values

# Interval de predicție 90%
min_pred = np.percentile(pred_array, 5, axis=1)
max_pred = np.percentile(pred_array, 95, axis=1)

print("\nPredictii:")
print("bani investiti -> interval vanzari (5 - 95)%")
for adv, l, h in zip(new, min_pred, max_pred):
    print(f"Publiciy = {adv * 1000} -> : {l * 1000} - {h * 1000}")


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, slope, dispersie]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 1 seconds.
Sampling: [y_obs]


Output()

Rezumat
            mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
intercept  3.377  0.218   2.949    3.781      0.004    0.003    2960.0   
slope      1.706  0.032   1.646    1.765      0.001    0.001    3018.0   

           ess_tail  r_hat  
intercept    3036.0    1.0  
slope        3039.0    1.0  

Coefficient HDI 94%
<xarray.Dataset> Size: 80B
Dimensions:    (hdi: 2)
Coordinates:
  * hdi        (hdi) <U6 48B 'lower' 'higher'
Data variables:
    intercept  (hdi) float64 16B 2.949 3.781
    slope      (hdi) float64 16B 1.646 1.765
Attributes:
    created_at:                 2025-12-03T13:05:18.691770+00:00
    arviz_version:              0.22.0
    inference_library:          pymc
    inference_library_version:  5.26.1
    sampling_time:              1.396000862121582
    tuning_steps:               1000



Predictii:
bani investiti -> interval vanzari (5 - 95)%
Publiciy = 2500.0 -> : 5215.026924405131 - 6674.554200023974
Publiciy = 5000.0 -> : 6038.547310652081 - 7524.291135040916
Publiciy = 9000.0 -> : 6582.574802435511 - 8032.091183895183
